In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import to_categorical

Скачивание данных

In [ ]:
images = np.load('images.npy')
labels = np.load('labels.npy')
test_images = np.load('images_sub.npy')

Нормализация данных

In [ ]:
images = images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

One-hot encoding

In [ ]:
labels = to_categorical(labels, num_classes=26)

One-hot encoding

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    images, labels, test_size=0.2, random_state=42)

Создание модели

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(26, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(
    monitor='val_accuracy', patience=5, restore_best_weights=True)

Обучение модели

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Предсказание модели и запись в файл

In [ ]:
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

submission = pd.DataFrame({
    'Id': range(len(predicted_classes)),
    'Category': predicted_classes
})
submission.to_csv('submission.csv', index=False)